In [68]:
# Ignore the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Data manipulation, visualization and useful functions
import argparse
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing

# gcp functions
from google.cloud import bigquery
from google.cloud import storage
import time
from datetime import datetime, timedelta
import pickle

In [69]:
n_steps=7

In [70]:
# Set up BigQuery clients
bqclient = bigquery.Client(project='charged-genre-350106')
storage_client = storage.Client(project='charged-genre-350106')


In [71]:
# Query for base dataset
query_tr = """
    SELECT *
    FROM `charged-genre-350106.demand_forecasting.airport_passenger_tr_nm_ns_7_lt21_tr`
    """

query_ts = """
    SELECT *
    FROM `charged-genre-350106.demand_forecasting.airport_passenger_ts_nm_ns_7_lt21_tr`
    """

In [72]:
trds = bqclient.query(query_tr).to_dataframe()
tsds = bqclient.query(query_ts).to_dataframe()

In [73]:
def datasplit_X_Y(data, Y_colname, X_colname):
    X_colname = [x for x in data.columns if x not in Y_colname + X_remove]
    Y_data = data[Y_colname]
    X_data = data[X_colname]
    print('X:', X_data.shape, 'Y:', Y_data.shape)
    return X_data, Y_data

In [74]:
# Data Split to X and Y
Y_colname = ['count']
X_remove = ['datetime', 'data_split']
X_colname = [x for x in trds.columns if x not in Y_colname + X_remove]
X_train, Y_train = datasplit_X_Y(trds, Y_colname, X_colname)
X_test, Y_test = datasplit_X_Y(tsds, Y_colname, X_colname)

X: (183, 420) Y: (183, 1)
X: (21, 420) Y: (21, 1)


In [75]:
continous_feature=[
'ticket_1', 'ticket_2', 'ticket_3','ticket_4','ticket_5','ticket_6','ticket_7',
                    'ticket_8', 'ticket_9', 'ticket_10','ticket_11','ticket_12','ticket_13','ticket_14',
                    'ticket_15','ticket_16', 'ticket_17','ticket_18','ticket_19','ticket_20','ticket_21',
                    'fare_min_1','fare_min_2','fare_min_3','fare_min_4','fare_min_5','fare_min_6','fare_min_7',
                    'fare_min_8','fare_min_9','fare_min_10','fare_min_11','fare_min_12','fare_min_13','fare_min_14',
                    'fare_min_15','fare_min_16','fare_min_17','fare_min_18','fare_min_19','fare_min_20','fare_min_21', 
                    'fare_max_1','fare_max_2','fare_max_3','fare_max_4','fare_max_5','fare_max_6','fare_max_7',
                    'fare_max_8','fare_max_9','fare_max_10','fare_max_11','fare_max_12','fare_max_13','fare_max_14',
                    'fare_max_15','fare_max_16','fare_max_17','fare_max_18','fare_max_19','fare_max_20','fare_max_21',
                    'fare_avg_1','fare_avg_2','fare_avg_3','fare_avg_4','fare_avg_5','fare_avg_6','fare_avg_7',
                    'fare_avg_8','fare_avg_9','fare_avg_10','fare_avg_11','fare_avg_12','fare_avg_13','fare_avg_14',
                    'fare_avg_15','fare_avg_16','fare_avg_17','fare_avg_18','fare_avg_19','fare_avg_20','fare_avg_21',
                    'fare_std_1','fare_std_2','fare_std_3','fare_std_4','fare_std_5','fare_std_6','fare_std_7',
                    'fare_std_8','fare_std_9','fare_std_10','fare_std_11','fare_std_12','fare_std_13','fare_std_14',
                    'fare_std_15','fare_std_16','fare_std_17','fare_std_18','fare_std_19','fare_std_20','fare_std_21',
                    'seat_min_1','seat_min_2','seat_min_3','seat_min_4','seat_min_5','seat_min_6','seat_min_7',
                    'seat_min_8','seat_min_9','seat_min_10','seat_min_11','seat_min_12','seat_min_13','seat_min_14',
                    'seat_min_15','seat_min_16','seat_min_17','seat_min_18','seat_min_19','seat_min_20','seat_min_21',
                    'seat_max_1','seat_max_2','seat_max_3','seat_max_4','seat_max_5','seat_max_6','seat_max_7',
                    'seat_max_8','seat_max_9','seat_max_10','seat_max_11','seat_max_12','seat_max_13','seat_max_14',
                    'seat_max_15','seat_max_16','seat_max_17','seat_max_18','seat_max_19','seat_max_20','seat_max_21',
                    'seat_avg_1','seat_avg_2','seat_avg_3','seat_avg_4','seat_avg_5','seat_avg_6','seat_avg_7',
                    'seat_avg_8','seat_avg_9','seat_avg_10','seat_avg_11','seat_avg_12','seat_avg_13','seat_avg_14',
                    'seat_avg_15','seat_avg_16','seat_avg_17','seat_avg_18','seat_avg_19','seat_avg_20','seat_avg_21',
                    'seat_std_1','seat_std_2','seat_std_3','seat_std_4','seat_std_5','seat_std_6','seat_std_7',
                    'seat_std_8','seat_std_9','seat_std_10','seat_std_11','seat_std_12','seat_std_13','seat_std_14',
                    'seat_std_15','seat_std_16','seat_std_17','seat_std_18','seat_std_19','seat_std_20','seat_std_21',
                    'b_1','b_2','b_3','b_4','b_5','b_6','b_7','b_8','b_9','b_10','b_11','b_12','b_13','b_14','b_15','b_16','b_17','b_18','b_19','b_20','b_21',
                    'f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8','f_9','f_10','f_11','f_12','f_13','f_14','f_15','f_16','f_17','f_18','f_19','f_20','f_21',
                    'd_1','d_2','d_3','d_4','d_5','d_6','d_7','d_8','d_9','d_10','d_11','d_12','d_13','d_14','d_15','d_16','d_17','d_18','d_19','d_20','d_21',
                    's_1','s_2','s_3','s_4','s_5','s_6','s_7','s_8','s_9','s_10','s_11','s_12','s_13','s_14','s_15','s_16','s_17','s_18','s_19','s_20','s_21',
                    'sd_1','sd_2','sd_3','sd_4','sd_5','sd_6','sd_7','sd_8','sd_9','sd_10','sd_11','sd_12','sd_13','sd_14','sd_15','sd_16','sd_17','sd_18','sd_19','sd_20','sd_21',
                      'lt_1',   'lt_2',   'lt_3',   'lt_4',   'lt_5',   'lt_6' , 'lt_7', 
                      'lt_8',   'lt_9',   'lt_10',  'lt_11',  'lt_12',  'lt_13', 'lt_14',
                      'lt_15',   'lt_16',   'lt_17',  'lt_18',  'lt_19',  'lt_20', 'lt_21',
                      'f330_1',   'f330_2',   'f330_3',   'f330_4',   'f330_5',   'f330_6' , 'f330_7',  
                      'f330_8',   'f330_9',   'f330_10',  'f330_11',  'f330_12',  'f330_13', 'f330_14',
                      'f330_15',  'f330_16',  'f330_17',  'f330_18',  'f330_19',  'f330_20', 'f330_21',
                      'f737_1',   'f737_2',   'f737_3',   'f737_4',   'f737_5',   'f737_6' , 'f737_7',  
                      'f737_8',   'f737_9',   'f737_10',  'f737_11',  'f737_12',  'f737_13', 'f737_14',
                      'f737_15',  'f737_16',  'f737_17',  'f737_18',  'f737_19',  'f737_20', 'f737_21',
                      'group_1',   'group_2',   'group_3',   'group_4',   'group_5',   'group_6' , 'group_7',  
                      'group_8',   'group_9',   'group_10',  'group_11',  'group_12',  'group_13', 'group_14',
                      'group_15',   'group_16',   'group_17',  'group_18',  'group_19',  'group_20', 'group_21',
                      'total_1',   'total_2',   'total_3',   'total_4',   'total_5',   'total_6' , 'total_7',  
                      'total_8',   'total_9',   'total_10',  'total_11',  'total_12',  'total_13', 'total_14',
                      'total_15',   'total_16',   'total_17',  'total_18',  'total_19',  'total_20', 'total_21',
                      'count_lag7','count_lag8','count_lag9','count_lag10','count_lag11','count_lag12','count_lag13'
]

category_feature=[
'holiday', 'long_h', 'b_long_h', 'inter_h', 'b_inter_h','first_m','b_first_m',
'dayofweek_dummy_0','dayofweek_dummy_1','dayofweek_dummy_2','dayofweek_dummy_3','dayofweek_dummy_4','dayofweek_dummy_5','dayofweek_dummy_6'
]

In [76]:
scaler_X_tr = preprocessing.MinMaxScaler()
scaler_Y_tr = preprocessing.MinMaxScaler()

def scaling(x_train, y_train, x_test, y_test, scaler_X_tr=scaler_X_tr,scaler_Y_tr=scaler_Y_tr):
    
    ## Scaling
    X_train_con = scaler_X_tr.fit_transform(x_train[continous_feature])
    X_train_cat = x_train[category_feature].to_numpy()
    X_train_scaled = np.concatenate((X_train_con, X_train_cat), axis=1)
    Y_train_scaled = scaler_Y_tr.fit_transform(y_train)

    X_test_con = scaler_X_tr.transform(x_test[continous_feature])
    X_test_cat = x_test[category_feature].to_numpy()
    X_test_scaled = np.concatenate((X_test_con, X_test_cat), axis=1)
    Y_test_scaled = scaler_Y_tr.transform(y_test)
    
    print('X_train_scaled:', X_train_scaled.shape, 'Y_train_gru:', Y_train_scaled.shape)
    print('X_test_scaled:', X_test_scaled.shape, 'Y_test_gru:', Y_test_scaled.shape)
    
    return X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled

In [77]:
def inverse_scaling(data, scaler_Y_tr=scaler_Y_tr):
    data_scaled = np.empty([data.shape[1], data.shape[0]])
    for i in range(data.shape[1]):
        data_scaled[i] = scaler_Y_tr.inverse_transform([data[:,i]])
    return data_scaled.transpose()

In [78]:
X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled = scaling(X_train, Y_train, X_test, Y_test)

X_train_scaled: (183, 420) Y_train_gru: (183, 1)
X_test_scaled: (21, 420) Y_test_gru: (21, 1)


In [79]:
X_train_scaled

array([[0.08273561, 0.10172975, 0.53279632, ..., 0.        , 0.        ,
        1.        ],
       [0.02218515, 0.04658332, 0.76869965, ..., 0.        , 0.        ,
        1.        ],
       [0.10558799, 0.13375578, 0.61449942, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.04588034, 0.17006337, 0.10011507, ..., 0.        , 0.        ,
        0.        ],
       [0.12677231, 0.13238568, 0.28138817, ..., 0.        , 0.        ,
        0.        ],
       [0.01634696, 0.04298681, 0.10586881, ..., 0.        , 0.        ,
        0.        ]])

In [80]:
def multi_input(x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled):
    ## X / Y Split for train
    X_train_multi_scaled, Y_train_multi_scaled = [], []
    for index in range(n_steps, len(x_train_scaled)+1):
        # find the end of this pattern
        end_ix = index 
        out_end_ix = end_ix -1
        # check if we are beyoind the dataset
        if out_end_ix > len(x_train_scaled):
            break
        # gather input and output part of the pattern
        seq_x, seq_y = x_train_scaled[index - n_steps: index, :], y_train_scaled[index - n_steps: index, -1]
        X_train_multi_scaled.append(seq_x)
        Y_train_multi_scaled.append(seq_y)
    X_train_multi_scaled, Y_train_multi_scaled = np.array(X_train_multi_scaled), np.array(Y_train_multi_scaled) 
    print('X_train_multi_scaled:', X_train_multi_scaled.shape, 'Y_train_multi_scaled:', Y_train_multi_scaled.shape)
    ## X / Y Split for test
    X_test_multi_scaled, Y_test_multi_scaled = [], []
    for index in range(n_steps, len(x_test_scaled)+1):
        # find the end of this pattern
        end_ix = index 
        out_end_ix = end_ix -1
        # check if we are beyoind the dataset
        if out_end_ix > len(x_test_scaled):
            break
        # gather input and output part of the pattern
        seq_x, seq_y = x_test_scaled[index - n_steps : index, :], y_test_scaled[index - n_steps : index, -1]
        X_test_multi_scaled.append(seq_x)
        Y_test_multi_scaled.append(seq_y)
    X_test_multi_scaled, Y_test_multi_scaled = np.array(X_test_multi_scaled), np.array(Y_test_multi_scaled) 
    print('X_test_multi_scaled:', X_test_multi_scaled.shape, 'Y_test_multi_scaled:', Y_test_multi_scaled.shape)
    
    return X_train_multi_scaled, Y_train_multi_scaled, X_test_multi_scaled, Y_test_multi_scaled

In [81]:
 X_train_multi_scaled, Y_train_multi_scaled, X_test_multi_scaled, Y_test_multi_scaled = multi_input(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled)

X_train_multi_scaled: (177, 7, 420) Y_train_multi_scaled: (177, 7)
X_test_multi_scaled: (15, 7, 420) Y_test_multi_scaled: (15, 7)


In [82]:
with open('X_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl', 'wb') as f:
    pickle.dump(X_test_multi_scaled,f)

with open('X_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl', 'wb') as f:
    pickle.dump(X_train_multi_scaled,f)

with open('Y_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl', 'wb') as f:
    pickle.dump(Y_test_multi_scaled,f)

with open('Y_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl', 'wb') as f:
    pickle.dump(Y_train_multi_scaled,f)

bucket = storage_client.get_bucket("mlops-test-kay")

blob = bucket.blob('airport_passenger/X_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')
blob.upload_from_filename('X_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')

blob = bucket.blob('airport_passenger/X_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')
blob.upload_from_filename('X_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')

blob = bucket.blob('airport_passenger/Y_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')
blob.upload_from_filename('Y_val_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')

blob = bucket.blob('airport_passenger/Y_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')
blob.upload_from_filename('Y_train_multi_scaled_nm_ns_7_lt21_tr_fm.pkl')